## Description

This notebook is to investigate how different convergence properties related to each other. The goal is to find the best properties defined that are materials independent and can be used to predict the convergence of a PP. 

The properties that are investigated are:
- For pressure, compare the complex defined SSSP v1 residue volume and the vannila hydrostatic pressure
- For EOS metrics, compare nu wrt AE and nu with ref 200Ry. (Check and assure the guess that delta' and nu are correlated)
- Compare pressure and EOS metrics (nu ref 200Ry)
- Other pair see if those are correlated or not

What I think is, if I tuning the criteria of properties, there will be a cross from A > B to B > A. The different between if A, B are correlated or not is whether their will be a state where A, B are highly linearly correlated. 

The testing data is generated by running full convergence test in the grid of [20:5:200] Ry for all different properties calculation method, then can extract and construct the properties date from the output.
The tested PPs are Hg, Ga, N, Cs, Mn from gbrv, dojo, psl-paw-high and jth, in order to cover PPs from different generated code sources and different type of elements.

This notebook is to compare that selected properties is or not dependent on the structure of the material.
The AiiDA data is stored at group `SI/convergence-properties-compare/<conf>`. conf is BCC/FCC/SC/DC 

In [1]:
from aiida import load_profile
import typing as t

load_profile("2023-08-07")

from aiida import orm

In [2]:
from aiida_sssp_workflow.workflows.convergence.pressure import helper_get_volume_from_pressure_birch_murnaghan
from aiida_sssp_workflow.calculations.calculate_bands_distance import get_bands_distance

def extract_data_scan_list1(node):
    pp_label = node.inputs.label.value.split(' ')[-1]
    
    real_scan_list = []
    for wf in node.called:
        if wf.process_label == 'ConvergencePressureWorkChain':
            lst = []
            for wf2 in wf.called:

                if wf2.process_label == 'helper_pressure_difference':
                    lst.append(wf2)
                if wf2.process_label == 'convergence_analysis':
                    break
            
            real_scan_list = wf.outputs.output_parameters_wfc_test.get_dict()['ecutwfc']
        
        else:
            # parse_pseudo_info or _CachingConvergenceWorkChain
            continue
        
    expected_scan_list = list(range(20, 201, 5))
    # find what is in expected but not in real
    missing = list(set(expected_scan_list) - set(real_scan_list))
    if missing:
        # raise a warning
        print(f"Warning - the following cutoffs are missing from node {node.pk}: {missing}")
        scan_list = real_scan_list
    else:
        scan_list = expected_scan_list

    data = {}

    for i, wf in enumerate(lst):
        cutoff = scan_list[i]

        i_para = wf.inputs.input_parameters.get_dict()
        r_para = wf.inputs.ref_parameters.get_dict()
        V0 = wf.inputs.V0.value
        B0 = wf.inputs.B0.value
        B1 = wf.inputs.B1.value
    
        # Get the data
        # Ref_200_nu: the nu value w.r.t. the 200 Ry reference
        # Ref_200_deltap: the delta_p value w.r.t. the 200 Ry reference
        res = get_conv_data1(i_para, r_para, V0, B0, B1)
        data[cutoff] = res

    return pp_label, data, scan_list

def get_conv_data1(i_para, r_para, V0, B0, B1) -> float:
    """Return AE_delta (per atom), AE_nu, REL_nu (the nu compare to the reference not to AE)"""
    res_pressure = i_para["hydrostatic_stress"]
    ref_pressure = r_para["hydrostatic_stress"]
    absolute_diff = abs(res_pressure - ref_pressure)
    relative_diff = helper_get_volume_from_pressure_birch_murnaghan(
        absolute_diff, V0, B0, B1,
    )

    return relative_diff

def extract_data_scan_list2(node):
    real_scan_list = []
    pp_label = node.inputs.label.value.split(' ')[-1]
    for wf in node.called:
        if wf.process_label == 'ConvergencePressureWorkChain':
            lst = []
            for wf2 in wf.called:

                if wf2.process_label == 'helper_pressure_difference':
                    lst.append(wf2)
                if wf2.process_label == 'convergence_analysis':
                    break
            
            real_scan_list = wf.outputs.output_parameters_wfc_test.get_dict()['ecutwfc']
        
        else:
            # parse_pseudo_info or _CachingConvergenceWorkChain
            continue
        
    expected_scan_list = list(range(20, 201, 5))
    # find what is in expected but not in real
    missing = list(set(expected_scan_list) - set(real_scan_list))
    if missing:
        # raise a warning
        print(f"Warning - the following cutoffs are missing from node {node.pk}: {missing}")
        scan_list = real_scan_list
    else:
        scan_list = expected_scan_list

    data = {}

    for i, wf in enumerate(lst):
        cutoff = scan_list[i]

        i_para = wf.inputs.input_parameters.get_dict()
        r_para = wf.inputs.ref_parameters.get_dict()
        V0 = wf.inputs.V0.value
        B0 = wf.inputs.B0.value
        B1 = wf.inputs.B1.value
    
        res = get_conv_data2(i_para, r_para)
        data[cutoff] = res

    return pp_label, data, scan_list

def get_conv_data2(i_para, r_para) -> float:
    """Return AE_delta (per atom), AE_nu, REL_nu (the nu compare to the reference not to AE)"""
    res_pressure = i_para["hydrostatic_stress"]
    ref_pressure = r_para["hydrostatic_stress"]
    absolute_diff = abs(res_pressure - ref_pressure)
    relative_on_simple_stress = absolute_diff / abs(ref_pressure)

    return relative_on_simple_stress


In [3]:
# from aiida_sssp_workflow.workflows.convergence.pressure import ConvergencePressureWorkChain
# from aiida_sssp_workflow.workflows.verifications import VerificationWorkChain

# qb = orm.QueryBuilder().append(
#     orm.Group,
#     filters={"label": {"==": "SI/convergence-properties-compare/SC"}},
#     tag="group",
# ).append(
#     VerificationWorkChain,
#     with_group="group",
#     tag="vc",
# ).append(
#     ConvergencePressureWorkChain,
#     with_incoming="vc",
#     tag="wc",
# ).append(
#     orm.CalcFunctionNode,
#     filters={
#         "attributes.function_name": "helper_pressure_difference",
#     },
#     tag="cf",
#     with_incoming="wc",
# ).append(
#     orm.Data,
#     with_outgoing="cf",
#     project=["*"],
# ).append(
#     orm.Dict,
#     with_incoming="wc",
#     project=["attributes.ecutwfc"],
# )

# qb.order_by({orm.CalcFunctionNode: {"ctime": "asc"}}).all()

In [4]:
g = 'SI/convergence-properties-compare/BCC'
gs_nodes = []
gs_nodes.extend(orm.Group.collection.get(label=g).nodes)
    
SC_all_data1 = {}
SC_all_data2 = {}
for node in gs_nodes:
    # give a node and the tuple of criteria
    # return the deducted cutoffs of A and B
    try:
        pp_label1, data1, scan_list1 = extract_data_scan_list1(node)
        pp_label2, data2, scan_list2 = extract_data_scan_list2(node)
        SC_all_data1[node.pk] = {
            "pp_label": pp_label1,
            "data": data1,
            "scan_list": scan_list1    
        }
        SC_all_data2[node.pk] = {
            "pp_label": pp_label2,
            "data": data2,
            "scan_list": scan_list2    
        }
    except Exception as e:
        #print(f"Error: {e}")
        #continue
        print(node.pk)
        raise e

In [5]:
g = 'SI/convergence-properties-compare/DC'
gs_nodes = []
gs_nodes.extend(orm.Group.collection.get(label=g).nodes)
    
DC_all_data1 = {}
DC_all_data2 = {}
for node in gs_nodes:
    # give a node and the tuple of criteria
    # return the deducted cutoffs of A and B
    try:
        pp_label1, data1, scan_list1 = extract_data_scan_list1(node)
        pp_label2, data2, scan_list2 = extract_data_scan_list2(node)
        DC_all_data1[node.pk] = {
            "pp_label": pp_label1,
            "data": data1,
            "scan_list": scan_list1    
        }
        DC_all_data2[node.pk] = {
            "pp_label": pp_label2,
            "data": data2,
            "scan_list": scan_list2    
        }
    except Exception as e:
        #print(f"Error: {e}")
        #continue
        print(node.pk)
        raise e

In [6]:
def extract_cutoff(data, scan_list, criteria):
    """Extract the cutoff for pA and pB from a verification workchain

    Args:
        data (dict): the data extracted from the verification workchain
        scan_list (list): the list of cutoffs used in the verification workchain
        criteria (tuple): first element is the criteria for pA, second element is the criteria for pB

    Returns:
        tuple: the cutoff for pA and pB.
    """
    # Get the cutoff of pA and pB
    cut = 200
    for cutoff in reversed(scan_list):
        try: 
            p = data[cutoff]
        except:
            continue
        
        if p > criteria:
            break

        cut = cutoff

    return cut

In [36]:
from operator import itemgetter

def compute_cutoff(data12_tuple, criteria):
    cut_A_lst = []
    cut_B_lst = []

    all_data1 = data12_tuple[0]
    all_data2 = data12_tuple[1]

    ## data1
    lst_label = []
    for node_pk in all_data1:
        pp_label = all_data1[node_pk]['pp_label']
        lst_label.append(pp_label)


    idx_sorted, pp_label_sorted = zip(*sorted(enumerate(lst_label), key=itemgetter(1)))
    sorted_pk_lst = [list(all_data1.keys())[i] for i in idx_sorted]

    for node_pk in sorted_pk_lst:
        data = all_data1[node_pk]['data']
        scan_list = all_data1[node_pk]['scan_list']
        cut_A = extract_cutoff(data, scan_list, criteria)
        cut_A_lst.append(cut_A)
    
    ## data2
    lst_label = []
    for node_pk in all_data2:
        pp_label = all_data2[node_pk]['pp_label']
        lst_label.append(pp_label)

    idx_sorted, pp_label_sorted = zip(*sorted(enumerate(lst_label), key=itemgetter(1)))
    sorted_pk_lst = [list(all_data2.keys())[i] for i in idx_sorted]

    for node_pk in sorted_pk_lst:
        data = all_data2[node_pk]['data']
        scan_list = all_data2[node_pk]['scan_list']
        cut_B = extract_cutoff(data, scan_list, criteria)
        cut_B_lst.append(cut_B)
        
    return cut_A_lst, cut_B_lst

In [37]:
# Get data for plotting (complex pressure)
SC_cut_A_lst, DC_cut_A_lst = compute_cutoff(data12_tuple=(SC_all_data1, DC_all_data1), criteria=0.1)

In [38]:
import ipywidgets as ipw
import plotly.graph_objects as go

trace_corr_scatter = go.Scatter(x=SC_cut_A_lst, y=DC_cut_A_lst, mode='markers', name='cutoff correlation')
trace_xy_line = go.Scatter(x=[0, 200], y=[0, 200], name='x=y')
g = go.FigureWidget(data=[trace_corr_scatter, trace_xy_line])
g.layout.xaxis.title = 'cutoff pA'
g.layout.yaxis.title = 'cutoff pB'


In [39]:
pA_slider = ipw.FloatSlider(value=0.1, min=0.00, max=10.0, step=0.01, description='pA')

def response(change):
    SC_cut_A_lst, DC_cut_A_lst = compute_cutoff(data12_tuple=(SC_all_data1, DC_all_data1), criteria=pA_slider.value)
    with g.batch_update():
        g.data[0].x = SC_cut_A_lst
        g.data[0].y = DC_cut_A_lst
        
pA_slider.observe(response, names="value")

slider_widgets = ipw.HBox([pA_slider])
app = ipw.VBox([slider_widgets, g])
app

In [33]:
# Get data for plotting (simple pressure)
SC_cut_B_lst, DC_cut_B_lst = compute_cutoff(data12_tuple=(SC_all_data2, DC_all_data2), criteria=0.1)

20
20


In [34]:
import ipywidgets as ipw
import plotly.graph_objects as go

trace_corr_scatter = go.Scatter(x=SC_cut_B_lst, y=DC_cut_B_lst, mode='markers', name='cutoff correlation')
trace_xy_line = go.Scatter(x=[0, 200], y=[0, 200], name='x=y')
g = go.FigureWidget(data=[trace_corr_scatter, trace_xy_line])
g.layout.xaxis.title = 'cutoff pA'
g.layout.yaxis.title = 'cutoff pB'

In [40]:
pB_slider = ipw.FloatSlider(value=0.1, min=0.00, max=10.0, step=0.1, description='pB')

def response(change):
    SC_cut_B_lst, DC_cut_B_lst = compute_cutoff(data12_tuple=(SC_all_data2, DC_all_data2), criteria=pA_slider.value)
    with g.batch_update():
        g.data[0].x = SC_cut_B_lst
        g.data[0].y = DC_cut_B_lst
        
pB_slider.observe(response, names="value")

slider_widgets = ipw.HBox([pA_slider])
app = ipw.VBox([slider_widgets, g])
app